# Differential Counts

I take code from 03.2017_SDSS_S82_Hess_diagrams,  to make all more manageable.

Anything here is meant to directly reproduce Fig.1  from Ivezic+2004 : differential counts of objects, selected by color cuts, or chi2 and color, cuts. 

## Read-in the data 

In [2]:
import pandas as pd
pd.options.display.max_columns = 999
from itertools import product
import numpy as np
import os 
import sys
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from matplotlib import rcParams

from scipy.stats import binned_statistic_2d
from astropy.table import Table
from astropy.table import vstack
from astropy.table import Column

import datetime

In [4]:
%time 
# NCSA 
# using astropy Table : discussion about speed 
# https://github.com/astropy/astropy/issues/3334 

# In the last resort : line-by-line ascii approach
# http://hea-www.harvard.edu/~aldcroft/tmp/p4a/hamogu/html/files/asciifiles.html
    
# old processing ( using old error even for faint objects, instead of 
# using the new faintRMS ).  
# fname1 =  'Var_ugriz_11_patches_NCSA_narrow.csv' 
# fname2 =  'Var_ugriz_11_patches_IN2P3_narrow.csv.gz' 

fname1 = 'VarD_ugriz_11_patches_NCSA_narrow_all.csv.gz'
fname2 = 'VarD_ugriz_11_patches_IN2P3_narrow_all.csv.gz'

DirIn = '../data_products/varMetricsMerged/'


nrows = 3e6
if nrows  : # if we want to limit the number of rows read in ... 
    # NCSA
    data1_df = pd.read_csv(DirIn+fname1, compression='gzip', 
                     nrows=nrows )
    data1 = Table.from_pandas(data1_df)
    # IN2P3
    data2_df = pd.read_csv(DirIn+fname2, compression='gzip', 
                     nrows=nrows )
    data2 = Table.from_pandas(data2_df)
    
else : # if no need to limit number of rows ... 
    data1 = Table.read(DirIn+fname1, format='csv')
    data2 = Table.read(DirIn+fname2, format='csv')

%time
# add a column saying which site is the data from ... 
new_col = Column(name='site',data= np.zeros_like(data1['patch']))
new_col[:] = 'NCSA'
data1.add_column(new_col)

# add a column saying which site is the data from ... 
new_col = Column(name='site',data= np.zeros_like(data2['patch']))
new_col[:] = 'IN2P3'
data2.add_column(new_col)

# Identical columns :
assert np.sum(np.in1d(data1.colnames, data2.colnames)) == len(data1.colnames)

# stack the two vertically ( rows over rows,  since the columns ARE IDENTICAL )
# this is even faster than any other merge, etc. 
data = vstack([data1, data2])

print('We have %d rows'%len(data))

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.77 µs
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.05 µs
We have 5748625 rows


## Plot the Fig.1 Ivezic+2004 : differential counts 

In [ ]:
# Find out what is the area by plotting the scatter of ra, dec 

plt.scatter(data['ra'][m], data['decl'][m], s=0.02)

In [ ]:
def plot_diff_counts(intercept):
    ''' A convenience function to plot the differential counts, 
    i.e. bottom two panels of Fig.1 of Ivezic+2004.  
    
    Input : 
    --------
    mag :  magnitude along which to plot the counts 
    
    
    '''

In [ ]:
def plot_all_panels(data=None, var=None, bounds=None, area=None):
    ''' A convenience function to plot all four panels 
    from Fig.1 Ivezic+2004. We use a function 
    plot_diff_counts() to plot the differential counts
    (bottom two panels). 
    
    Input: 
    ------
    data :  an AstroPy Table that contains columns with  {u,g,r}+'psfMean_all' , 
    'g'+{chi2DOF, chi2R}+'_all', i.e. all data to plot . Default : None
    
    var : a string that defines how we should cut on variability. NOTE : 
    this applies to ALL the data before plotting, not just selecting QSO! 
    Possible choices include  's0',  's0a', 's1', 's2a', 's2b' : all these
    denote different regions in chi2R - chi2DOF space . Default : None. 
    If var= None, we select only by color.  
    
    bounds : a float or tuple of floats,  that defines chi2 bounds needed 
    for different regions.  For 's0', 's1', 's2a',  bounds is a float, 
    by default = 2 .   For 's0a', it is a tuple of floats  : [1,1.5],  
    [1.5,2] are possible choices
    
    intercept :  a pair of floats allowing to move the diagonal lines 
    on bottom two plots (differential counts) up or down.  It is a tuple
    [intercept1,  intercept2], where the first value defines the intercept 
    for the left line (with slope 0.75), and the second value : for the 
    right line (with slope 0.30) - see Ivezic+2004 Fig.1 for explanation. 
    
    area : area of the patch of the sky from which the data comes , 
    to calculate differential counts [ in square degrees]. 
    
    Returns : 
    ------
    None
    '''
    # make aliases for the data ... 
    
    chi2R = data['gchi2R'+suffix].data.data
    chi2DOF = data['gchi2DOF'+suffix].data.data
    extendedness = data['extendedness'].data.data
    g = data['gpsfMean'+suffix].data.data
    r = data['rpsfMean'+suffix].data.data
    u = data['upsfMean'+suffix].data.data

    
    # general cuts : magnitude and color  
    mask_ext = extendedness == e 
    mask_ug = (-0.5 < (u-g)) * ((u-g) < 1.5) 
    mask_u  = u < 23.5
    mask_color = mask_ug * mask_u 
   
    print('The area is %.3f sq.degrees'%area)  
    
    title_color = 
    
    if var :  
        # add variability  cuts ... 
        if var is 's0' :
            
            title_var = 
            mask_var = 
            
        if var is 's0a' : 
            
        if var is 's1' : 
            
        if var is 's2a':  
            
            
            
        if var is 's2b' : 
        
        
        m = mask_color * mask_ext * mask_var 
    else :
        m = mask_color * mask_ext 
    
    nObj = np.sum(m)
    print('We have %d objects to start with '%nObj)
    
    # initialize plotting space 
    fig,axs = plt.subplots(2,2,figsize=(12,12))
    ax = np.ravel(axs)
    # plot settings : linewidth, linestyle, density and dash length ,
    # dot size 
    lw = 2; ls = '--' ; dashes = (5, 5); size=0.002
    
    
    # plot upper left panel  : u-g vs u 
    ug = u[m]-g[m]
    gr = g[m]-r[m]
    u = u[m]
    ax[0].scatter(ug,u, s=size)
    ax[0].invert_yaxis()
    ax[0].set_xlabel('u-g')
    ax[0].set_ylabel('u')
    # plot the two vertical lines at u=0.6 and 0.8 
    ax[0].axvline(0.6, lw=lw, ls=ls, c='r', dashes=dashes)
    ax[0].axvline(0.8, lw=lw, ls=ls, c='r', dashes=dashes)

    # plot upper right panel : u-g vs g-r 
    ax[1].scatter(ug,gr, s=size)
    ax[1].set_ylim(-0.6, 0.7)
    # plot the two vertical lines at u=0.6 and 0.8 
    ax[1].axvline(0.6, lw=lw, ls=ls, dashes=dashes)
    ax[1].axvline(0.8, lw=lw, ls=ls, dashes=dashes)
    ax[1].set_xlabel('u-g')
    ax[1].set_ylabel('g-r')
    # get the lims to plot the horizontal line 
    x1,x2 = ax[1].get_xlim()
    
    
    
    # plot bottom left and bottom right : diff counts 
    # in u and r 
    
    # assemble all masks together 
    # settings for the diff counts
    masks_diff_counts = [mask_qso, mask_trans, mask_hots]
    
    
    for mag, label, ax in zip([u[m], r[m]], ['u','r'], [ax[2], ax[3]]):
        plot_diff_counts(mag, )
        
        
    colors = ['red', 'green', 'blue']
    symbols = ['o', '^', 's'] # https://matplotlib.org/api/markers_api.html
    sizes = [10,6,6]
    dm = 0.2
    nbins   = np.arange(17,24,dm)    
        
   
    
   
    
    
    # plot a separate panel with diff counts in g  
    
    
    
        